<a href="https://colab.research.google.com/github/mahunz/Diplodatos2020/blob/master/MARCE%20Analisis_y_Curacion_Grupo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h2>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h2>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2020</h3>
<h3>Predicción de la Calidad de Servicio</h3>
<h3>Análisis y Curación</h3>
</center>
</left>
<h4>Mentor: Martín Hunziker</h4>

[Link Mentoria](https://sites.google.com/view/mentorias2020-diplodatos/ciencia-de-datos-aplicada-en-la-distribuci%C3%B3n-de-energ%C3%ADa-el%C3%A9ctrica?authuser=0).

</left>
</left>

#Grupo 2
###Marcelo Tisera
###Laura Aliendo
###Joel de Andres

### Introducción

En la siguiente notebook, se presentará la consigna a seguir para el segundo práctico de la materia Análisis y Curación. El objetivo consiste en vincular las tablas del dataset, así como iniciar el tratamiento de los datos geográficos.

Como referencia para el análisis geográfico utilizaremos la notebook 2_0_Intro_Variables_georefrenciadas de Ramiro Caro.


In [ ]:
# Instalación de librerias en Collab
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install geopandas
!apt install libspatialindex-dev
!pip install rtree
!pip install osmnx
!pip install contextily

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [ ]:
# En Collab
!wget -nc https://github.com/mahunz/Diplodatos2020/blob/master/Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb.7z?raw=true
!apt-get install p7zip
!mkdir -p Dataset
!mv ENF_6612_2018-12-31_M10_20190529-1610.gdb.7z?raw=true ./Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb.7z
!7za x ./Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb.7z
!mv ENF_6612_2018-12-31_M10_20190529-1610.gdb ./Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb

--2020-09-29 00:17:10--  https://github.com/mahunz/Diplodatos2020/blob/master/Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb.7z?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/mahunz/Diplodatos2020/raw/master/Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb.7z [following]
--2020-09-29 00:17:10--  https://github.com/mahunz/Diplodatos2020/raw/master/Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb.7z
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mahunz/Diplodatos2020/master/Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb.7z [following]
--2020-09-29 00:17:10--  https://raw.githubusercontent.com/mahunz/Diplodatos2020/master/Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb.7z
Resolving raw.githubusercontent.com (raw.githubuserconten

In [ ]:
# En Collab
!mkdir -p DatasetOrigin
!cp -i -r ./Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb ./DatasetOrigin/ENF_6612_2018-12-31_M10_20190529-1610_origin.gdb

cp: overwrite './DatasetOrigin/ENF_6612_2018-12-31_M10_20190529-1610_origin.gdb/ENF_6612_2018-12-31_M10_20190529-1610.gdb/ENF_6612_2018-12-31_M10_20190529-1610.gdb/a00000001.freelist'? 

### I. Rutina de Curación

Inicialmente, con el objetivo de preparar los datos que alimentarán futuros modelos de aprendizaje automático (ML), se propone seguir la siguiente __[checklist](https://dimewiki.worldbank.org/wiki/Checklist:_Data_Cleaning)__ para la limpieza de los datos de nuestro proyecto. Esta _checklist_ es la misma que utilizaron en el primer práctico de la materia y nos será de utilidad como guía para curar el dataset. A modo de ayuda, **en esta notebook encontrarán una especie de template** que sigue la _checklist_ y que deberán ir completando.

**Cada decisión tomada deberá quedar registrada de manera explícita y clara.** Luego de pasar por todos los puntos de la _checklist_ propuesta, deberán almacenar en un nuevo archivo los datos resultantes.


### II. Análisis del Contenido

Una vez aplicado el  _Checklist_, lo que vamos a hacer es comenzar a vincular las diferentes tablas del dataset e iniciar la generación de features geográficas


# Resolución

## I. Rutina de Curación

### 1. Importación de Datos

#### 1.1. Verificación de Inexistencia de Problemas en la Importación

A partir de este punto, utilizaremos los transformadores de distribucion como elemnto de analisis tabla UNTRD, e incorporaremos datos de consumidores, subestación y líneas de distribución.

Para comenzar, importamos los datos que vamos a procesar de las diferentes tablas que vamos a analizar:

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString
import numpy as np


In [ ]:
# Editar con el path correspondiente a sus datos
data_path = './Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb'
data_path_origin = './Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb'

print(data_path)
print(data_path_origin)
 

In [ ]:
print(data_path)
untrd = gpd.read_file(data_path, driver='FileGDB', layer='UNTRD')
untrd.shape, untrd.columns

In [ ]:
untrs = gpd.read_file(data_path, driver='FileGDB', layer='UNTRS')
untrs.shape, untrs.columns

In [ ]:
ssmt = gpd.read_file(data_path, driver='FileGDB', layer='SSDMT')
ssmt.shape, ssmt.columns

In [ ]:
segcon = gpd.read_file(data_path, driver='FileGDB', layer='SEGCON')
segcon.shape, segcon.columns

In [ ]:
ucbt = gpd.read_file(data_path, driver='FileGDB', layer='UCBT')
ucbt.shape, ucbt.columns

In [ ]:
ucbt_cod_id = ucbt.reset_index()
ucbt_cod_id['COD_ID'] = ucbt_cod_id.index

In [ ]:
# Agrupando Variables

ucbt['ENE_MED'] = ucbt[['ENE_01', 'ENE_02', 'ENE_03', 'ENE_04', 'ENE_05', 'ENE_06', 'ENE_07','ENE_08', 'ENE_09', 'ENE_10', 'ENE_11', 'ENE_12']].mean(axis=1)
ucbt['ENE_STD'] = ucbt[['ENE_01', 'ENE_02', 'ENE_03', 'ENE_04', 'ENE_05', 'ENE_06', 'ENE_07','ENE_08', 'ENE_09', 'ENE_10', 'ENE_11', 'ENE_12']].std(axis=1)
ucbt['ENE_MAX'] = ucbt[['ENE_01', 'ENE_02', 'ENE_03', 'ENE_04', 'ENE_05', 'ENE_06', 'ENE_07','ENE_08', 'ENE_09', 'ENE_10', 'ENE_11', 'ENE_12']].max(axis=1)


ucbt_cod_id['ENE_MED'] = ucbt_cod_id[['ENE_01', 'ENE_02', 'ENE_03', 'ENE_04', 'ENE_05', 'ENE_06', 'ENE_07','ENE_08', 'ENE_09', 'ENE_10', 'ENE_11', 'ENE_12']].mean(axis=1)
ucbt_cod_id['ENE_STD'] = ucbt_cod_id[['ENE_01', 'ENE_02', 'ENE_03', 'ENE_04', 'ENE_05', 'ENE_06', 'ENE_07','ENE_08', 'ENE_09', 'ENE_10', 'ENE_11', 'ENE_12']].std(axis=1)
ucbt_cod_id['ENE_MAX'] = ucbt_cod_id[['ENE_01', 'ENE_02', 'ENE_03', 'ENE_04', 'ENE_05', 'ENE_06', 'ENE_07','ENE_08', 'ENE_09', 'ENE_10', 'ENE_11', 'ENE_12']].max(axis=1)

### 1.2. Asegurar la Existencia de IDs o Claves Únicas

In [ ]:
print(untrd['COD_ID'].is_unique)
print(untrs['COD_ID'].is_unique)
print(ssmt['COD_ID'].is_unique)
print(segcon['COD_ID'].is_unique)

El siguiente paso implica chequear que no existen datos duplicados y que las claves, si existen, son únicas.

Las claves únicas de las diferentes tablas es el COD_ID, como figura en la documentación. En el caso de la tabla UCBT fue eliminado por el proveedor de los datos por lo que asumiremos que los datos son únicos. Podriamos regenerar el COD_ID con un valor secuencial, pero como vamos a utilizar la tabla UNTRD como elementos de análisis no será necesario

In [ ]:
cod_id = 'COD_ID'
untrd_duplicado_cod_id  = untrd[untrd[cod_id].duplicated(keep=False)]
untrd_duplicado_cod_id

In [ ]:
untrs_duplicado_cod_id  = untrs[untrs[cod_id].duplicated(keep=False)]
untrs_duplicado_cod_id

In [ ]:
ssmt_duplicado_cod_id  = ssmt[ssmt[cod_id].duplicated(keep=False)]
ssmt_duplicado_cod_id

In [ ]:
segcon_duplicado_cod_id  = segcon[segcon[cod_id].duplicated(keep=False)]
segcon_duplicado_cod_id

In [ ]:
ucbt_duplicado_cod_id  = ucbt_cod_id[ucbt_cod_id[cod_id].duplicated(keep=False)]
ucbt_duplicado_cod_id

### 1.3. Despersonalizar Datos y Guardarlos en un Nuevo Archivo

En este caso, no es necesario despersonalizar ningún dato, dado que los mismos ya se encuentran despersonalizados.

### 1.4. Nunca Modificar los Datos Crudos u Originales

Al finalizar la limpieza, deberán guardar el dataset resultante, para asegurarse de no modificar los datos originales.

Creamos 2 dataset para evitar modificar el original 1 con el nombre de que esta en un dir DatasetOrigin/data_path_origin.

El que usaremos para modificar que es el data_path en el dir Dataset/

## 2. Pasos de Limpieza Necesarios

### 2.1. Etiquetas de Variables/Columnas y Problemas de Codificación/Encoding

El geopandas automáticamente detecta el encoding de los datos. 

### 2.2. Tratamiento de Valores Faltantes

Para analizar los valores faltantes, primero deberán saber cuántos existen por campo y cuánto representan del total:

In [ ]:
# get the number of missing data points per column
list_dataset = [{"name":"UNTRD","data_set":untrd},
{"name":"UNTRS","data_set":untrs},
{"name":"SSMT","data_set":ssmt}, 
{"name":"SEGCON","data_set":segcon}, 
{"name":"UCBT","data_set":ucbt_cod_id}]

for dataset in list_dataset:
  missing_value = dataset.get("data_set").isnull().sum()
  count_value = dataset.get("data_set").COD_ID.count()
  print("\nValores null de {} : {}".format(dataset.get("name"),missing_value[missing_value > 0]))
  if len( missing_value[missing_value > 0]) > 0 :
    for i in missing_value[missing_value > 0]:
      print("Cantidad de registros  : {}".format(count_value))
      print("Cantidad de registros Null : {}".format(i))
      print("Representa el % : {}".format((i*100)/count_value))
  else:
    print("No tiene valores Nulls")



En la tabla de SEGCON existen valores null en el campo geometry en la totalidad de los registros.

### 2.3. Codificación de Variables Categóricas

Aplica?

In [ ]:
from sklearn import preprocessing
datasets = {'untrd': untrd, 'ucbt': ucbt}
columns_to_encode = ['ARE_LOC', 'GRU_TAR']

for key, dataset in datasets.items():
    for column in columns_to_encode:
        if column in dataset:
            le = preprocessing.LabelEncoder()
            le.fit(dataset[column])
            dataset[column] = le.transform(dataset[column])

### 2.4. No Cambiar los Nombres de las Variables de la Fuente de Origen

### 2.5. Verificación de Consistencia de Datos

Este es el paso más analítico, en donde se deben aplicar reglas de integridad.

Si se eliminana lo valores NaN que pasa con los registros de segcon? 

In [ ]:
untrd = untrd.dropna()
untrs = untrs.dropna()
ssmt = ssmt.dropna()
segcon = segcon.dropna()
ucbt = ucbt.dropna()

### 2.6. Identificar y Documentar Valores Atípicos/Outliers

Calcular estadísticos.

In [ ]:
def find_outliers(dataset, column_name):
    interesting_col = dataset[column_name]
    CI =  (interesting_col.quantile(0.75)-interesting_col.quantile(0.25))
    mask_outlier = (
        np.abs(interesting_col - interesting_col.median()) >= (1.5 * CI))
    return dataset[mask_outlier]


main_statisticians = ['FIC', 'DIC', 'ENE_MED']

outliers = []
for key, dataset in datasets.items():
    print("Dataset: ", key)
    for column in main_statisticians:
        if column in dataset:
            outliers = find_outliers(dataset, column)
            row_quantity = dataset.shape[0]
            print("Cantidad de outliers para la variable ", column, len(outliers))
    print("------------------------------------------------")

In [ ]:
# Procedemos a limpiar los outliers para la layer ucbt
def clean_outliers(dataset, column_name):
    interesting_col = dataset[column_name]
    CI =  (interesting_col.quantile(0.75)-interesting_col.quantile(0.25))
    mask_outlier = (
        np.abs(interesting_col - interesting_col.median()) <= (1.5 * CI))
    return dataset[mask_outlier]


for column in main_statisticians:
    ucbt_without_outliers = clean_outliers(ucbt, column)
datasets['ucbt'] = ucbt_without_outliers


### 2.7. Evaluar Cómo Comprimir los Datos Para su Almacenamiento Más Eficiente

### 2.8. Guardar el Set de Datos con un Nombre Informativo

## 3. Pasos de Limpieza Deseables

### 3.1. Ordenar Variables/Columnas

### 3.2. Quitar Variables/Columnas Irrelevantes

Tenemos alguna en este caso?

In [ ]:
datasets['ucbt'].drop(datasets['ucbt'].columns.difference(['GRU_TAR', 'UNI_TR_D', 'UNI_TR_S', 'DIC', 'FIC', 'ENE_MED', 'ENE_MAX']), 1, inplace=True)

In [ ]:
datasets['untrd'].drop(datasets['untrd'].columns.difference(['COD_ID', 'UNI_TR_S', 'ARE_LOC', 'geometry']), 1, inplace=True)

### 3.3. Renombrar Variables de Grillas

### 3.4. Categorizar Variables que Contengan “Otros”

### 3.5. Agregar Metadata a los Datos

Cuando y como fueron obtenidos, limpieza realizada, decisiones implementadas, asunciones, etc.

## II. Análisis  Contenido

### 1. Features Tipo Objetos

In [ ]:
import scipy as sc
datasets['ucbt'] = datasets['ucbt'].groupby(['UNI_TR_D']).agg(GRU_TAR=('GRU_TAR',lambda x: sc.stats.mode(x)[0]),
                                                              DIC=('DIC',np.sum),FIC=('FIC',np.sum),ENE_MED=('ENE_MED',np.mean),
                                                              ENE_MAX=('ENE_MAX',np.sum)).reset_index()
datasets['ucbt']

    -GRU_TAR: tomamos su moda
    -DIC: calculamos su suma
    -FIC: calculamos su suma
    -ARE_LOC: tomamos su moda
    -ENE_MED: calculamos su promedio
    -ENE_MAX: calculamos su suma

## Consumindores

1 - A partir de los resultados obtenidos en el práctico 1 seleccione que features podrían ser relevantes para la predicción de la calidad de servicio.

2 - Agrupe los datos en base a la variable UNI_TR_D.

3 - Justifique los métodos utilizados para agrupar las variables numéricas y categóricas.

In [ ]:
#med_group = ucbt.groupby("UNI_TR_D").agg({"DIC":"mean", "FIC":"mean"})
ucbt_group = ucbt.groupby("UNI_TR_D").agg(GRU_TAR=('GRU_TAR',lambda x: sc.stats.mode(x)[0]),
                                                              DIC=('DIC',np.sum),FIC=('FIC',np.sum),ENE_MED=('ENE_MED',np.mean),
                                                              ENE_MAX=('ENE_MAX',np.sum)).reset_index()
ucbt_group                                                              

In [ ]:
untrd_merge = untrd.merge(ucbt_group,left_on='COD_ID',right_on='UNI_TR_D',how="left")

In [ ]:
untrd_merge[untrd_merge.DIC.isnull()]

## Subestación

4 - Analise la información disponible sobre los transformadores de las Subestaciones (UNI_TR_S > UNTRS)

5 - Seleccione las variables relevantes, realize tratamiento de valores invalidos y outliers correspondientes. Agrupe los valores por UNTRD y agregue al dataset.

In [ ]:
ucbt_group

In [ ]:
untrd.to_crs(epsg=31984, inplace= True)
untrs.to_crs(epsg=31984, inplace= True)

In [ ]:
untrd_ucbt_group = ucbt_group.merge(untrd,left_on="UNI_TR_D", right_on="COD_ID")
untrd_ucbt_group = untrd_ucbt_group.drop(['COD_ID'], axis=1)
untrd_ucbt_group.rename(columns={"geometry": "geometry_untrd"}, inplace=True)
untrd_ucbt_group


In [ ]:
untrs_cp = untrs.copy()
untrs_cp.to_crs(epsg=31984, inplace= True)

In [ ]:
untrs_cp

In [ ]:
#de UNTRS solo tomamos el id y geometry.
copy_untrs = untrs[['COD_ID','geometry']]
copy_untrs

In [ ]:
group_ucbt_untrd_untrs = untrd_ucbt_group.merge(copy_untrs, left_on='UNI_TR_S', right_on='COD_ID')
group_ucbt_untrd_untrs = group_ucbt_untrd_untrs.drop(['COD_ID'], axis=1)
group_ucbt_untrd_untrs.rename(columns={"geometry": "geometry_untrs"}, inplace=True)
group_ucbt_untrd_untrs

## Generación de variables geográficas

5 - Calcule la distancia euclideana de cada untrd al correspondiente untrs y adicione al dataset.

6 - Cree una variable que sea la distancia al centro urbano mas cercanos.

7 - Usando OSMnx descargue la informacion sobre caminos dentro del area de cobertura de la distribuidora. Cree variables para las distancias a autopistas (trunk), a caminos primarios (primary) y a caminos secundarios (secondary)


In [ ]:
import osmnx 

In [ ]:
group_ucbt_untrd_untrs['EUC_DIST_TRUNK']=-1
group_ucbt_untrd_untrs['EUC_DIST_PRIMARY']=-1
group_ucbt_untrd_untrs['EUC_DIST_SECUNDARY']=-1

In [ ]:
group_ucbt_untrd_untrs['EUC_DIST_UNTRS'] = -1



In [ ]:

for i,t in group_ucbt_untrd_untrs.iterrows():
    dist = group_ucbt_untrd_untrs.geometry_untrd[i].distance(group_ucbt_untrd_untrs.geometry_untrs[i])
    group_ucbt_untrd_untrs.EUC_DIST_UNTRS.loc[i] = dist

In [ ]:
group_ucbt_untrd_untrs

In [ ]:
centro_urbano = Point(136050.57090952771250159, 7531008.69087591022253036)
group_ucbt_untrd_untrs['EUC_DIST_CENT'] = -1
for i,t in group_ucbt_untrd_untrs.iterrows():
    eud = group_ucbt_untrd_untrs.geometry_untrd[i].distance(centro_urbano)
    group_ucbt_untrd_untrs.EUC_DIST_CENT.loc[i] = eud
    
group_ucbt_untrd_untrs 

## [Opcional] Generación de variables con arboles de busqueda

8 - Cree variables que indique la cantidad de consumidores en radios de 25, 50 y 100 metros del transformador.

9 - Cree una variable indicando la distancia media de cada fila (cada untrd) a los 5 untrd mas cercanos.

10 - Usando OSMnx descargue los footprints de edificios disponibles, y cree una variable que indique la cantidad de edificios en un radio de 200 metros al transformador.

In [ ]:
group_ucbt_untrd_untrs[group_ucbt_untrd_untrs.EUC_DIST_UNTRS<=33877]